In [17]:
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.neighbors import BallTree
from sklearn.neighbors import KDTree
from sklearn.neighbors import DistanceMetric
import pandas as pd
import numpy as np
from tqdm import tqdm
seed = 1

## Jaccard index

In [1]:
from math import*
  
def jaccard_similarity(x,y):
    
    intersection_cardinality = len(set.intersection(*[set(x), set(y)]))
    union_cardinality = len(set.union(*[set(x), set(y)]))
    return intersection_cardinality/float(union_cardinality)
  
print (jaccard_similarity([0,1,2,5,6],[0,2,3,5,7,9]))

0.375


In [8]:
quantity_map = pd.read_csv('ingredients_quantity_map.csv', ',',  index_col='id')

In [9]:
bitmap = pd.read_csv('../data/clean_ingredients_bitmap.csv', ',',  index_col='id')

In [ ]:
pairwise_distances(bitmap[:10,:], metric='jaccard', n_jobs=-1)

In [131]:
bitmap_matrix = bitmap.as_matrix()
bitmap_matrix = bitmap_matrix.astype(np.float16)

In [10]:
recipe_ids = list(quantity_map.index)

## Get jaccard recipe pairs

In [136]:
ingredients_dict={}
for id in recipes_data.index:
    for ingredient in recipes_data.at[id, 'ingredients']:
        ingredients_dict.update({int(ingredient['id']):ingredient['text']})


In [ ]:
from sklearn.neighbors import BallTree


In [ ]:
%%time
tree = BallTree(bitmap, metric='jaccard')              

In [ ]:
%%time
dist, ind = tree.query(bitmap, k=11) 


In [ ]:
np.save('distances_top_11', dist)
np.save('indices_top_11', ind)

In [14]:
dist = np.load('distances_top_11.npy')
ind = np.load('indices_top_11.npy')

In [1322]:
recipes = pd.read_json('../data/recipes_clean.json')

In [547]:
ingredients = set([ingredient['id'] for ingredients in recipes['ingredients'] for ingredient in ingredients])

In [ ]:
ingredients_dict={}
for id in recipes.index:
    for ingredient in recipes.at[id, 'ingredients']:
        ingredients_dict.update({int(ingredient['id']):ingredient['text']})


In [ ]:
bitmap = pd.read_csv('../data/clean_ingredients_bitmap.csv', ',',  index_col='id')
bitmap = bitmap.astype(np.float16)

In [ ]:
quantity_map = pd.read_csv('ingredients_quantity_map.csv', ',',  index_col='id')
#quantity_map = quantity_map.astype(np.float16)

In [1305]:
quantity_map_normalized = quantity_map.div(quantity_map.sum(axis=1), axis=0)

In [1282]:
quantity_map_normalized.loc[100008].iloc[quantity_map_normalized.loc[100008].nonzero()]

1526     113.333333
18752      6.300000
18868    340.000000
4343     226.400000
Name: 100008, dtype: float64

In [1268]:
recipes_pruned = np.load('recipes_pruned.npy')
recipes_pruned = [recipe for recipe in recipes_pruned.tolist()]

In [18]:
ingredients_pruned = np.load('ingredients_pruned.npy')

In [1306]:
remove = [ingredient for ingredient in ingredients if ingredient not in ingredients_pruned]

In [1358]:
qmp =  quantity_map.drop(labels=remove, axis='columns')

In [1307]:
qmnp = quantity_map.drop(labels=remove, axis='columns')
qmnp = qmnp.div(qmnp.sum(axis=1), axis=0)

In [ ]:
dist = np.load('distances_top_11.npy')
ind = np.load('indices_top_11.npy')

In [1323]:
id2position={}
for i, id_ in enumerate(recipes.index):
    id2position[id_]=i

position2id = {v: k for k, v in id2position.items()}

In [1313]:
## with recipe_id
def get_ingredients(recipe_id):
    return qmnp.loc[recipe_id][qmnp.loc[recipe_id]!=0].index.tolist()


In [ ]:
## with recipe_position
def get_ingredients(recipe_ord):
    return qmnp.loc[qmnp.index[recipe_ord]][qmnp.loc[qmnp.index[recipe_ord]]!=0].index.tolist()


In [1315]:
def count_recipe_connections(recipe_1, recipe_2):
    vector_1 = set(get_ingredients(recipe_1))
    vector_2 = set(get_ingredients(recipe_2))
    #vector_1 = set(build_ingredients_vector(recipe_1))
    #vector_2 = set(build_ingredients_vector(recipe_2))
    mutual_ingredients = vector_1.intersection(vector_2)
    in_recipe1 = vector_1-mutual_ingredients
    in_recipe2 = vector_2-mutual_ingredients
    #print('score 1: {} score 2: {}'.format(get_score(recipe_1), get_score(recipe_2)))
    return mutual_ingredients, in_recipe1, in_recipe2

In [1324]:
from collections import Counter

def find_pairs():
    ing2variance = {}
    for ingredient in ingredients_pruned:
        ing2variance.update({ingredient:[]})
    count = 0
    pair_dict = {}
    to_add = Counter()
    to_remove = Counter()
    no_change = Counter()
    base = Counter()
    for i, recipe in tqdm(enumerate(qmnp.index)):#bitmap_matrix.shape[0]):
        #build_recipe_vector(bitmap_matrix.index[recipe], unit_converter)
        for j, similar_recipe in enumerate(ind[i]):
            if recipe != position2id[similar_recipe]:
                mutual, in_1, in_2 = count_recipe_connections(recipe, position2id[similar_recipe])
                
                for ing in mutual:
                    ing2variance[ing].append(qmnp.loc[recipe][str(ing)])
                if len(in_1) == 0 or len(in_2) == 0:
                    count+=1
                    
                    if len(in_1) != 0:
                        tmp_ing =  in_1.copy()
                        in_1 =  in_2 
                        in_2 = tmp_ing
                    
                    key = sorted([recipe,position2id[similar_recipe]])
                    pair_dict['{}:{}'.format(key[0], key[1])]={'base':mutual, 'extra':in_2, 'distance': dist[i,j]}

                    
                        #added_score[ingredient].append(hs_1-hs_2)
                        #track_recipe_addition.

                    #show_recipe_connections(recipe, similar_recipe)

                    #print('\n****\n')
            
    return pair_dict, ing2variance

In [1325]:
pairs, ing2variance = find_pairs()

41797it [13:58, 49.82it/s]


In [1328]:
np.save('pairs', pairs)

In [1362]:
diff = {}
for ing in ingredients_pruned:
    diff.update({ing:[]})
for item in tqdm(pairs):
    r1 = int(item[:item.index(':')])
    r2 = int(item[item.index(':')+1:])
    
    for ing in pairs[item]['base']:
        diff[ing].append(np.abs(qmp.loc[r1][str(ing)]-qmp.loc[r2][str(ing)]))
    for ing in pairs[item]['extra']:
        try: 
            diff[ing].append(qmp.loc[r1][str(ing)])
        except:
            diff[ing].append(qmp.loc[r2][str(ing)])
    


100%|██████████| 24088/24088 [14:28<00:00, 27.72it/s]


In [1365]:
max_diff = 0
for ing in diff:
    if max([dif for dif in diff[ing]])>max_diff:
        max_diff = max([dif for dif in diff[ing]])
        ing_max = ing

In [1378]:
sum_dict = {}
for ing in diff:
    sum_dict[ing] = sum([dif for dif in diff[ing]])


In [1383]:
import operator
sorted_x = sorted(sum_dict.items(), key=operator.itemgetter(1), reverse=True)

In [1390]:
for ing,_ in sorted_x[:20]:
    print(ingredients_dict[int(ing)])

2 1/2 cups all-purpose flour
1 cup white sugar
2 tablespoons water
2 tablespoons milk
1 cup butter or margarine, melted
2 cups confectioners' sugar
1 cup packed brown sugar
2 potatoes, chopped
3 cups bread flour
1 1/2 cups semisweet chocolate chips
ice cubes
3 tablespoons heavy whipping cream
1/2 cup shortening
1 medium apple, cored and thinly sliced
1 cup chopped walnuts (optional)
3/4 cup rolled oats
3 cups shredded Cheddar cheese
2 cups fresh strawberries
2 tablespoons lemon juice
2 tablespoons olive oil
